### querying data in audit load logs delta table

In [1]:
%%sql
select * from BronzeLH.load_logs

StatementMeta(, 739b93bc-1e12-4106-9f3f-45389bf97314, 2, Finished, Available, Finished)

<Spark SQL result set with 22 rows and 5 fields>

### DDL for creating table transactions_b in Bronze lakehouse in pysparksql

In [34]:
%%sql
CREATE TABLE transactions_b (
    TransactionID STRING NOT NULL,
    EncounterID STRING NOT NULL,
    PatientID STRING NOT NULL,
    ProviderID STRING NOT NULL,
    DeptID STRING NOT NULL,
    VisitDate date NOT NULL,
    ServiceDate date NOT NULL,
    PaidDate date NOT NULL,
    VisitType STRING NOT NULL,
    Amount float NOT NULL,
    AmountType STRING NOT NULL,
    PaidAmount float NOT NULL,
    ClaimID STRING NOT NULL,
    PayorID STRING NOT NULL,
    ProcedureCode int NOT NULL,
    ICDCode STRING NOT NULL,
    LineOfBusiness STRING NOT NULL,
    MedicaidID STRING NOT NULL,
    MedicareID STRING NOT NULL,
    InsertDate date NOT NULL,
    ModifiedDate date NOT NULL
)USING DELTA;


StatementMeta(, 57b9d0ff-540d-4d5c-9052-965db64d4d96, 36, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

### DDL for creating table transactions_a in Bronze lakehouse in pysparksql

In [ ]:
%%sql
CREATE TABLE transactions_a (
    TransactionID STRING NOT NULL,
    EncounterID STRING NOT NULL,
    PatientID STRING NOT NULL,
    ProviderID STRING NOT NULL,
    DeptID STRING NOT NULL,
    VisitDate date NOT NULL,
    ServiceDate date NOT NULL,
    PaidDate date NOT NULL,
    VisitType STRING NOT NULL,
    Amount float NOT NULL,
    AmountType STRING NOT NULL,
    PaidAmount float NOT NULL,
    ClaimID STRING NOT NULL,
    PayorID STRING NOT NULL,
    ProcedureCode int NOT NULL,
    ICDCode STRING NOT NULL,
    LineOfBusiness STRING NOT NULL,
    MedicaidID STRING NOT NULL,
    MedicareID STRING NOT NULL,
    InsertDate date NOT NULL,
    ModifiedDate date NOT NULL
)USING DELTA;

### DDL for creating table patients_b in Bronze lakehouse in pysparksql

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS BronzeLH.patients_b (
    PatientID STRING,
    FirstName STRING,
    LastName STRING,
    MiddleName STRING,
    SSN STRING,
    PhoneNumber STRING,
    Gender STRING,
    DOB DATE,
    Address STRING,
    ModifiedDate TIMESTAMP
)
USING DELTA;

### DDL for creating table patients_a in Bronze lakehouse in pysparksql

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS BronzeLH.patients_a (
    PatientID STRING,
    FirstName STRING,
    LastName STRING,
    MiddleName STRING,
    SSN STRING,
    PhoneNumber STRING,
    Gender STRING,
    DOB DATE,
    Address STRING,
    ModifiedDate TIMESTAMP
)
USING DELTA;

### create audit load logs delta table in Bronze lakehouse

In [14]:
%%sql

CREATE TABLE IF NOT EXISTS BronzeLH.load_logs (
    
    data_source STRING,
    tablename STRING,
    numberofrowscopied INT,
    watermarkcolumnname STRING,
    loaddate TIMESTAMP
)USING DELTA;

StatementMeta(, 57b9d0ff-540d-4d5c-9052-965db64d4d96, 16, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

### DDL for creating table departments_a in Bronze lakehouse in pysparksql

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS BronzeLH.departments_a (
DeptId string,
Name string
 )
USING DELTA;

### DDL for creating table departments_b in Bronze lakehouse in pysparksql

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS BronzeLH.departments_b (
DeptId string,
Name string
 )
USING DELTA;

### DDL for creating table encounters_b in Bronze lakehouse in pysparksql

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS BronzeLH.encounters_b (
EncounterID string,
PatientID string,
EncounterDate date,
EncounterType string,
ProviderID string,
DepartmentID string,
ProcedureCode integer,
InsertedDate date,
ModifiedDate date
)
USING DELTA;

### DDL for creating table encounters_a in Bronze lakehouse in pysparksql

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS BronzeLH.encounters_a (
EncounterID string,
PatientID string,
EncounterDate date,
EncounterType string,
ProviderID string,
DepartmentID string,
ProcedureCode integer,
InsertedDate date,
ModifiedDate date
)
USING DELTA;

### DDL for creating table providers_a in Bronze lakehouse in pysparksql

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS BronzeLH.providers_a (
ProviderID string,
FirstName string,
LastName string,
Specialization string,
DeptID string,
NPI long
)
USING DELTA;

### DDL for creating table providers_b in Bronze lakehouse in pysparksql

In [ ]:
%%sql
CREATE TABLE IF NOT EXISTS BronzeLH.providers_b (
ProviderID string,
FirstName string,
LastName string,
Specialization string,
DeptID string,
NPI long
)
USING DELTA;

#### Read claims parquet format data from ADLSGen2 write to BronzeLH in delta table format

In [41]:
from pyspark.sql.functions import lit,col
df1=spark.read.parquet("Files/project/bronze/hospital1_claim_data")#accessing using shortcut in fabric
df2=spark.read.parquet("Files/project/bronze/hospital2_claim_data")
#display(df1)
df_source1=df1.withColumn("data_source",lit("hosa"))
df_source2=df2.withColumn("data_source",lit("hosb"))
df_union=df_source1.unionByName(df_source2)
df_union.write.format('delta').saveAsTable('claim_data')
#display(df_source1.head(5))
#display(df_source2.head(5))
#df.write.format('delta')

StatementMeta(, 5e9d9efe-7f23-45eb-a446-80a130e70d91, 43, Finished, Available, Finished)

#### Read cptcodes parquet format data from ADLSGen2 write to BronzeLH in delta table format

In [16]:
from pyspark.sql.functions import *
df_cpt_code_rename=spark.read.parquet("Files/project/bronze/cptcode")
#display(df)
for col in df_cpt_code_rename.columns:
    col_rename=col.replace(" ","_").lower()
    df_cpt_code_rename=df_cpt_code_rename.withColumnRenamed(col,col_rename)
df_cpt_code_rename.createOrReplaceTempView('cpt_rename')
display(df_cpt_code_rename)
#display(type(df_cpt_code_rename))
df_cpt_code_rename.write.format("delta").saveAsTable("cpt_codes")


StatementMeta(, 5e9d9efe-7f23-45eb-a446-80a130e70d91, 18, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3beebb91-87db-43d6-8912-9045591ef717)

### Querying tables

In [5]:
#df1=spark.read.table('departments_a')
#display(df1.head(3))
#df2=spark.read.table('departments_b')
#display(df2.head(3))
df3=spark.read.table('providers_a')
display(df3.head(3))

StatementMeta(, 982429e8-8ecd-4b4e-876b-156403cef91e, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 61c10268-bad3-44dd-a941-08fb5479b34b)

#### write patients data into bronze lakehouse in delta format

In [8]:
df_patients_a=spark.read.parquet("Files/project/bronze/hos-a/patients_a")
df_patients_a.write.format("delta").mode('append').saveAsTable("patients_a")
df_patients_b=spark.read.parquet("Files/project/bronze/hos-b/patients_b")
df_patients_b.write.format("delta").mode('append').saveAsTable("patients_b")
#display(df_patients_a.head(3))

StatementMeta(, 982429e8-8ecd-4b4e-876b-156403cef91e, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, bd6099f7-e3f5-4266-9798-3c7d5a11d5d3)

#### write encounters data into bronze lakehouse in delta format

In [ ]:
df_encounters_a=spark.read.parquet("Files/project/bronze/hos-a/encounters_a")
df_encounters_a.write.format("delta").mode('append').saveAsTable("encounters_a")
df_encounters_b=spark.read.parquet("Files/project/bronze/hos-b/encounters_b")
df_encounters_b.write.format("delta").mode('append').saveAsTable("encounters_b")

#### write providers data into bronze lakehouse in delta format

In [ ]:
df_providers_a=spark.read.parquet("Files/project/bronze/hos-a/providers_a")
df_providers_a.write.format("delta").mode('append').saveAsTable("providers_a")
df_providers_b=spark.read.parquet("Files/project/bronze/hos-b/providers_b")
df_providers_b.write.format("delta").mode('append').saveAsTable("providers_b")

#### write transactions data into bronze lakehouse in delta format

In [ ]:
df_transactions_a=spark.read.parquet("Files/project/bronze/hos-a/transactions_a")
df_transactions_a.write.format("delta").mode('append').saveAsTable("transactions_a")
df_transactions_b=spark.read.parquet("Files/project/bronze/hos-b/transactions_b")
df_transactions_b.write.format("delta").mode('append').saveAsTable("transactions_b")

#### write departments data into bronze lakehouse in delta format

In [ ]:
df_departments_a=spark.read.parquet("Files/project/bronze/hos-a/departments_a")
df_departments_a.write.format("delta").mode('append').saveAsTable("departments_a")
df_departments_b=spark.read.parquet("Files/project/bronze/hos-b/departments_b")
df_departments_b.write.format("delta").mode('append').saveAsTable("departments_b")